In [1]:
import pandas as pd
import re
import string
import nltk
import math
from nltk.tokenize import sent_tokenize
from textblob import TextBlob
from nltk.stem.snowball import SnowballStemmer
from spellchecker import SpellChecker
from textblob import Word
from wordcloud import STOPWORDS
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
sns.set()

In [2]:
desc = pd.read_csv('DataInDeed.csv')

In [3]:
desc

,Unnamed: 0,title,salary,description
0,0,Service Delivery Manager – Well Intervention (...,NaN,Service Delivery Manager – Well Intervention (...
1,1,Associate Risk Assurance (IT Risk),NaN,Line of Service\nAssurance\nIndustry/Sector\nN...
2,2,"Database Administrator (SQL, Oracle)",NaN,"Database Administrator (SQL, Oracle)\nTimo\nỨn..."
3,3,Senior HRIS Executive,NaN,Ninja Van is a tech-enabled logistics company ...
4,4,Key Account Manager,NaN,Design solutions to drive safe living and qual...
...,...,...,...,...
1001,655,Country Business Director - Power Tools Division,NaN,Company Description\n\nThe Bosch Group is a le...
1002,656,Flutter Developer,NaN,"Job Description\nParticipate in analysis, desi..."
1003,657,AWS Cloud Solutions Architect,NaN,Khối / Ban\nKhối Dịch vụ Công nghệ Ngân hàng\n...
1004,658,SOFTWARE ENGINEER (JAVA),NaN,Responsibilities and Duties\n\nDevelopment on ...


In [4]:
stop_words = set(stopwords.words('english'))

In [17]:
example_sent = desc['description'][1]

In [8]:
word_tokens = word_tokenize(example_sent)

In [16]:
desc['title'][0]

'Service Delivery Manager – Well Intervention (Vietnam)'

In [70]:
#they are present in stop_words or not
filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
#with no lower case conversion
filtered_sentence = []

for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)
  
print(len(filtered_sentence))

481


https://realpython.com/sentiment-analysis-python <br>
https://towardsdatascience.com/getting-started-with-text-nlp-visualization-9dcb54bc91dd

In [5]:
def splitData(name, newName, stringSplit, df):
    data = df[['Unnamed: 0', name]]

    # phân chuỗi trong cột skill ra list rồi thực chia theo cột unnamed: 0
    skill = pd.DataFrame(data.set_index('Unnamed: 0')[name].str.split(stringSplit, expand=True).stack().reset_index(level=1, drop=True)).reset_index()

    # đổi tên cột 0 trong dataFrame skill thành 'skill_cut'
    skill.rename(columns = {0:newName}, inplace = True)

    # gộp dataFrame df và skill vói nhay theo "Unnameed: 0"
    df = df.merge(skill, left_on = 'Unnamed: 0', right_on = 'Unnamed: 0')
    return df

In [6]:
desc = splitData('description', 'split_description', '\n\n', desc)

In [7]:
desc.split_description[1000]

'Qualifications\nBachelor or Master degree major in Electronic, Mechatronic, Mechanical Engineering, Automotive Engineering.\nAutomotive projects experience\nEmbedded system\nSW development processes\nAutomotive knowledge\nFMEA methodology\nEnthusiastic, motivated, critical thinking & result oriented\nGood discipline, positive thinking, teamwork\nGood in communication skills and above (including English and soft skill) & good in Japanese is a big plus'

In [11]:
def replaceString(string):
    if (string.rfind('(') >=0) & (string.rfind(')') >=0):
        indexStr1 = string.index('(')
        indexStr2 = string.index(')')
        char = string[indexStr1: indexStr2+1]
        string = string.replace(char, '')
    if (string.rfind('[') >= 0) & (string.rfind(']') >= 0):
        indexStr3 = string.index('[')
        indexStr4 = string.index(']')
        char1 = string[indexStr3: indexStr4+1]
        string = string.replace(char1, '')
    return string





def missValue(string):
    textVaul = ''
    string = string.split(" ")
    for word in string:
        gfg = Word(word)
        result = gfg.spellcheck()[0][0]
        textVaul += result
        textVaul += " "
    textVaul = textVaul.strip()
    return textVaul


def missSpellSting(textvaul):
    
    new_doc = TextBlob(textvaul)

    correctResult = new_doc.correct()
    result  = missValue(correctResult)
    return str(result)


def removeString(string):
    string = string.lower()
    string = replaceString(string)
    string = string.replace('\n', ' ')
    string = re.sub('\W', ' ', string)
    
    
    string = string.strip()
    string = missSpellSting(string)
    return string
    

In [12]:
removeString('Service Delivery Manager – Well Intervention (Vietnam)')

'service delivery manager   well intervention'

In [ ]:
desc['split_description'] = desc['split_description'].apply(lambda x: removeString(x))